In [1]:
import pandas as pd
import numpy as np
import keras
import tensorflow
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense


data = pd.read_csv("train.txt", sep=';')
data.columns = ["Text", "Emotions"]
print(data.head())

                                                Text Emotions
0  i can go from feeling so hopeless to so damned...  sadness
1   im grabbing a minute to post i feel greedy wrong    anger
2  i am ever feeling nostalgic about the fireplac...     love
3                               i am feeling grouchy    anger
4  ive been feeling a little burdened lately wasn...  sadness


In [2]:
texts = data["Text"].tolist()
labels = data["Emotions"].tolist()

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

In [3]:
sequences = tokenizer.texts_to_sequences(texts)
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length)

In [4]:
# Encode the string labels to integers
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [5]:
# One-hot encode the labels
one_hot_labels = keras.utils.to_categorical(labels)

In [6]:
# Split the data into training and testing sets
xtrain, xtest, ytrain, ytest = train_test_split(padded_sequences, 
                                                one_hot_labels, 
                                                test_size=0.2)

In [7]:
# Define the model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, 
                    output_dim=128, input_length=max_length))
model.add(Flatten())
model.add(Dense(units=128, activation="relu"))
model.add(Dense(units=len(one_hot_labels[0]), activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(xtrain, ytrain, epochs=10, batch_size=32, validation_data=(xtest, ytest))

Epoch 1/10
400/400 [==============================] - 24s 57ms/step - loss: 1.3330 - accuracy: 0.4946 - val_loss: 0.8182 - val_accuracy: 0.7256
Epoch 2/10
400/400 [==============================] - 22s 56ms/step - loss: 0.3225 - accuracy: 0.8986 - val_loss: 0.5232 - val_accuracy: 0.8331
Epoch 3/10
400/400 [==============================] - 24s 60ms/step - loss: 0.0531 - accuracy: 0.9871 - val_loss: 0.5521 - val_accuracy: 0.8288
Epoch 4/10
400/400 [==============================] - 23s 56ms/step - loss: 0.0243 - accuracy: 0.9945 - val_loss: 0.5689 - val_accuracy: 0.8366
Epoch 5/10
400/400 [==============================] - 22s 56ms/step - loss: 0.0172 - accuracy: 0.9961 - val_loss: 0.5997 - val_accuracy: 0.8338
Epoch 6/10
400/400 [==============================] - 23s 58ms/step - loss: 0.0133 - accuracy: 0.9971 - val_loss: 0.6887 - val_accuracy: 0.8288
Epoch 7/10
400/400 [==============================] - 25s 62ms/step - loss: 0.0135 - accuracy: 0.9967 - val_loss: 0.6652 - val_accuracy:

In [10]:
input_text = "This is so awesome. I can't believe this happened."

# Preprocess the input text
input_sequence = tokenizer.texts_to_sequences([input_text])
padded_input_sequence = pad_sequences(input_sequence, maxlen=max_length)
prediction = model.predict(padded_input_sequence)
predicted_label = label_encoder.inverse_transform([np.argmax(prediction[0])])
print(predicted_label)

1/1 [==============================] - 0s 37ms/step
['joy']
